In [ ]:
!pip install langchain langchain_community
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes accelerate
!pip install llama-cpp-python


In [1]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [9]:

llm_cpp = LlamaCpp(
            streaming = True,
            model_path="/kaggle/input/siemens-langchain/zephyr-7b-beta.Q4_0.gguf",
            n_gpu_layers=4,
            n_batch=512,
            temperature=0.6,
            top_p=1,
            verbose=True,
            n_ctx=4096
            )

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /kaggle/input/siemens-langchain/zephyr-7b-beta.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

## Prompt

In [ ]:

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)


In [8]:
from langchain.chains import LLMChain


prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm_cpp, prompt=prompt_template_name, output_key="restaurant_name")


prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm_cpp, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
chain({"cuisine": "Indian"})

In [ ]:
#!pip install streamlit

In [ ]:
dir(chain)

## SerpAPI

In [ ]:
import os
#!pip install google-search-results

os.environ['SERPAPI_API_KEY'] = "27c61505166ceb4302b1e07f6169cf412a042c29d301e449d56b1f781cf5abb0"

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools


# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm_cpp)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm_cpp, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors = True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

In [6]:
prompt_template_name

PromptTemplate(input_variables=['cuisine'], template='I want to open a restaurant for {cuisine} food. Suggest one name only.')

## Memory

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm_cpp, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)


llama_print_timings:        load time =    3588.12 ms
llama_print_timings:      sample time =     144.87 ms /   256 runs   (    0.57 ms per token,  1767.05 tokens per second)
llama_print_timings: prompt eval time =    3588.00 ms /    20 tokens (  179.40 ms per token,     5.57 tokens per second)
llama_print_timings:        eval time =  100642.71 ms /   255 runs   (  394.68 ms per token,     2.53 tokens per second)
llama_print_timings:       total time =  105506.17 ms /   275 tokens




We will suggest you some fency names for your Mexican food restaurant based on the type of cuisine served, the location, and the target audience. Here are our suggestions:

1. Estancia Azteca - This name combines the Spanish word "estancia," which means estate or place, with "Azteca" to represent the rich cultural heritage of Mexico's indigenous people. It has a sophisticated and elegant sound that would appeal to an upscale clientele.

2. La Hacienda de Oro - This name translates to "The Golden Ranch" in Spanish, which evokes images of luxury and opulence. The word "hacienda" also has connotations of tradition and history, making it a fitting name for a Mexican restaurant that serves authentic dishes with a modern twist.

3. Tequila Luna - This name combines the words "tequila," which is a popular Mexican spirit, with "luna," which means moon in Spanish. It has a sophisticated and elegant sound that would appeal to an upscale clientele, and it also conveys a sense of mystery and all

In [13]:
print(chain.memory.buffer)

Human: Mexican
AI: 

We will suggest you some fency names for your Mexican food restaurant based on the type of cuisine served, the location, and the target audience. Here are our suggestions:

1. Estancia Azteca - This name combines the Spanish word "estancia," which means estate or place, with "Azteca" to represent the rich cultural heritage of Mexico's indigenous people. It has a sophisticated and elegant sound that would appeal to an upscale clientele.

2. La Hacienda de Oro - This name translates to "The Golden Ranch" in Spanish, which evokes images of luxury and opulence. The word "hacienda" also has connotations of tradition and history, making it a fitting name for a Mexican restaurant that serves authentic dishes with a modern twist.

3. Tequila Luna - This name combines the words "tequila," which is a popular Mexican spirit, with "luna," which means moon in Spanish. It has a sophisticated and elegant sound that would appeal to an upscale clientele, and it also conveys a sense

## CHATBOT

In [15]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm_cpp)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [16]:
convo.run("Who won the first cricket world cup?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3588.12 ms
llama_print_timings:      sample time =     143.34 ms /   256 runs   (    0.56 ms per token,  1785.94 tokens per second)
llama_print_timings: prompt eval time =   13488.59 ms /    70 tokens (  192.69 ms per token,     5.19 tokens per second)
llama_print_timings:        eval time =   98864.50 ms /   255 runs   (  387.70 ms per token,     2.58 tokens per second)
llama_print_timings:       total time =  113647.33 ms /   325 tokens


" Blessed with a rich cricketing history dating back over two centuries, the sport of cricket has witnessed some exhilarating moments in international tournaments. The first-ever ICC Cricket World Cup was held in England in 1975. The tournament featured eight teams, namely, Australia, Bangladesh (then East Pakistan), England, India, New Zealand, Pakistan, South Africa, and the West Indies.\n\nAfter a round-robin group stage, the teams were divided into two groups based on their points tally. Group A comprised of Australia, New Zealand, Pakistan, and the West Indies, while Group B featured England, India, South Africa, and Bangladesh (then East Pakistan).\n\nThe Super Sixes stage followed, where each team played against the other three teams from a different group. The top two teams from this stage progressed to the semi-finals.\n\nIn the semi-finals, Australia defeated New Zealand by 47 runs, while the West Indies beat Pakistan by four wickets. In the final, held at the iconic Lord's C

In [18]:
# restricting convo buffer size
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
convo = ConversationChain(llm=llm_cpp, memory=memory)
convo.run("5+5")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3588.12 ms
llama_print_timings:      sample time =     138.62 ms /   256 runs   (    0.54 ms per token,  1846.74 tokens per second)
llama_print_timings: prompt eval time =    1398.58 ms /     7 tokens (  199.80 ms per token,     5.01 tokens per second)
llama_print_timings:        eval time =   98632.78 ms /   255 runs   (  386.80 ms per token,     2.59 tokens per second)
llama_print_timings:       total time =  101322.07 ms /   262 tokens


' The sum of five and five is ten. Ten is a number that comes after nine in a sequence called counting, and before twelve in the same sequence. It is also a digit used to represent the number ten when writing numerals in certain number systems such as decimal system. In binary notation, it is written as 1010. In ternary notation, it is written as 102. In quaternary notation, it is written as 23.\nIn mathematics, ten is the base used in the decimal system, which is also referred to as denary system or base-ten numeral system. It is commonly used for counting and measuring because it has just enough digits to be convenient for everyday use while still being small enough to be handled by the human mind without major difficulties.\nIn many languages, the word for ten in native numerals is derived from Latin decem, with cognates such as German zehn, Dutch tien, Danish tyve, Norwegian ti, Swedish tio, French dix, Romanian zece, Polish dziesięć, Lithuanian desimtis, Latvian desmit, Estonian k